# Interactive Map with Drawing Tools

This notebook demonstrates how to create an interactive map using Folium, allowing users to draw shapes directly on the map. Drawn shapes can be retrieved and converted into GeoPandas/Shapely objects for further geospatial analysis.

In [1]:
# Import Required Libraries
import folium
from folium.plugins import Draw
import geopandas as gpd
from shapely.geometry import shape
import ipywidgets as widgets
from IPython.display import display, Javascript, clear_output

In [3]:
# Create and display a Folium map with drawing tools
m = folium.Map(location=[55.6761, 12.5683], zoom_start=6)  # Centered on Denmark
Draw(export=True).add_to(m)
m

## Instructions
- Use the drawing tools on the map to create polygons, rectangles, or other shapes.
- Click the 'Export' button to download the drawn shapes as a GeoJSON file.
- The next cell will show how to load the exported GeoJSON and convert it to GeoPandas/Shapely objects.

In [ ]:
# Load exported GeoJSON and convert to GeoPandas/Shapely objects
import json
from shapely.geometry import shape
import geopandas as gpd

# Upload the exported GeoJSON file
uploader = widgets.FileUpload(accept='.geojson', multiple=False)
display(uploader)

# Function to process uploaded file
def process_geojson(uploaded):
    if uploaded.value:
        content = list(uploaded.value.values())[0]['content']
        geojson_data = json.loads(content.decode())
        # Convert to GeoPandas GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
        print('GeoDataFrame:')
        display(gdf)
        # Convert to Shapely geometries
        geometries = [shape(feature['geometry']) for feature in geojson_data['features']]
        print('Shapely Geometries:')
        for geom in geometries:
            print(geom)

uploader.observe(lambda change: process_geojson(uploader), names='value')

In [2]:
# Automatically save drawn shapes from Folium map to work folder
import os
from IPython.display import Javascript, display
import ipywidgets as widgets

save_path = os.path.join(os.getcwd(), 'drawn_shapes.geojson')
output = widgets.Output()
display(output)

js_code = f"""
require(['base/js/namespace'], function(Jupyter) {{
    var mapFrame = document.querySelector('iframe');
    if (!mapFrame) return;
    var mapDoc = mapFrame.contentDocument || mapFrame.contentWindow.document;
    var exportBtn = mapDoc.querySelector('.leaflet-draw-toolbar a[title="Export"]');
    if (!exportBtn) return;
    exportBtn.addEventListener('click', function() {{
        setTimeout(function() {{
            var geojsonText = '';
            var links = mapDoc.querySelectorAll('a');
            links.forEach(function(link) {{
                if (link.download && link.download.endsWith('.geojson')) {{
                    fetch(link.href)
                        .then(response => response.text())
                        .then(text => {{
                            geojsonText = text.replace(/`/g, '\\`');
                            var pyCode = `geojson_data = '''${{geojsonText}}'''
with open(r\"{save_path}\", \"w\") as f:
    f.write(geojson_data)
print('GeoJSON saved to: {save_path}')`;
                            Jupyter.notebook.kernel.execute(pyCode);
                        }});
                }}
            }});
        }}, 1000);
    }});
}});
"""
display(Javascript(js_code))

print(f"When you click 'Export' on the map, the GeoJSON will be saved to: {save_path}")

Output()

<IPython.core.display.Javascript object>

When you click 'Export' on the map, the GeoJSON will be saved to: n:\dowf\drawn_shapes.geojson


In [1]:
# Interactive drawing with ipyleaflet and direct GeoJSON access
from ipyleaflet import Map, DrawControl
import geopandas as gpd
from shapely.geometry import shape
import json
import os

# Create map centered on Denmark
m = Map(center=(55.6761, 12.5683), zoom=6)
draw_control = DrawControl()
m.add_control(draw_control)
display(m)

# Container for drawn features
drawn_features = []

def handle_draw(target, action, geo_json):
    print(f"Action: {action}")
    print(f"GeoJSON: {geo_json}")
    drawn_features.append(geo_json)
    # Save to file automatically
    save_path = os.path.join(os.getcwd(), 'drawn_shapes_ipyleaflet.geojson')
    with open(save_path, 'w') as f:
        json.dump({'type': 'FeatureCollection', 'features': drawn_features}, f)
    print(f"Saved to {save_path}")
    # Convert to GeoPandas
    gdf = gpd.GeoDataFrame.from_features(drawn_features)
    display(gdf)
    # Convert to Shapely geometries
    geometries = [shape(feature['geometry']) for feature in drawn_features]
    print('Shapely Geometries:')
    for geom in geometries:
        print(geom)

draw_control.on_draw(handle_draw)

Map(center=[55.6761, 12.5683], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [2]:
import ipywidgets as widgets
widgets.Widget.widget_types

C:\Users\ge94faj\AppData\Local\Temp\ipykernel_21868\1311618427.py:2: DeprecationWarning: Widget.widget_types is deprecated.
  widgets.Widget.widget_types


In [ ]:
# Interactive display of GeoDataFrame
gdf = None
try:
    import geopandas as gpd
    gdf = gpd.read_file('drawn_shapes_ipyleaflet.geojson')
    try:
        import qgrid
        qgrid_widget = qgrid.show_grid(gdf, show_toolbar=True)
        display(qgrid_widget)
    except ImportError:
        print('qgrid not installed, showing standard table.')
        display(gdf)
except Exception as e:
    print('Error loading GeoDataFrame:', e)

In [ ]:
# Correct qgrid usage for interactive GeoDataFrame display
!pip install qgrid --quiet
import geopandas as gpd
import qgrid
qgrid_widget = None
try:
    gdf = gpd.read_file('drawn_shapes_ipyleaflet.geojson')
    qgrid_widget = qgrid.QgridWidget(df=gdf, show_toolbar=True)
    display(qgrid_widget)
except Exception as e:
    print('Error displaying interactive GeoDataFrame:', e)
    if 'gdf' in locals():
        display(gdf)

In [3]:
# Interactive map display of GeoDataFrame using gdf.explore()
import geopandas as gpd
try:
    gdf = gpd.read_file('drawn_shapes_ipyleaflet.geojson')
    m = gdf.explore()
    display(m)
except Exception as e:
    print('Error displaying interactive map:', e)

Error displaying interactive map: drawn_shapes_ipyleaflet.geojson: No such file or directory


In [ ]:
# Robust interactive map display of GeoDataFrame using gdf.explore()
import geopandas as gpd
try:
    gdf = gpd.read_file('drawn_shapes_ipyleaflet.geojson')
    # Remove any style argument, just use defaults
    m = gdf.explore()
    display(m)
    print('If you see an empty map, check that your GeoJSON contains valid geometries.')
except Exception as e:
    print('Error displaying interactive map:', e)
    if 'gdf' in locals():
        print('GeoDataFrame columns:', gdf.columns)
        display(gdf)

In [ ]:
# Add markers to a Folium map in a for loop and display it
import folium

# Example coordinates for markers
marker_coords = [
    (52.789, 12.129),
    (53.199, 14.678),
    (51.686, 13.733),
    (52.5, 13.0)
]

# Create map centered on average location
center_lat = sum([lat for lat, lon in marker_coords]) / len(marker_coords)
center_lon = sum([lon for lat, lon in marker_coords]) / len(marker_coords)
m = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# Add markers in a for loop
for lat, lon in marker_coords:
    folium.Marker(location=[lat, lon]).add_to(m)

# Display the map
m

In [ ]:
# Dynamically update 5 random markers on a map using ipyleaflet
import random
import time
from ipyleaflet import Map, Marker, LayerGroup
from IPython.display import display, clear_output

# Define bounds for random coordinates (e.g., within Denmark)
lat_min, lat_max = 54.5, 57.8
lon_min, lon_max = 8.0, 12.7

# Create map centered on Denmark
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2
m = Map(center=(center_lat, center_lon), zoom=6)
display(m)

layer_group = LayerGroup()
m.add_layer(layer_group)

def random_coords(n):
    return [(random.uniform(lat_min, lat_max), random.uniform(lon_min, lon_max)) for _ in range(n)]

for _ in range(10):  # Update 10 times as a demo
    layer_group.clear_layers()
    coords = random_coords(5)
    for lat, lon in coords:
        marker = Marker(location=(lat, lon))
        layer_group.add_layer(marker)
    time.sleep(1)  # Wait 1 second before updating